In [1]:
import pandas as pd
import os
import detect_simpsons_paradox as dsp
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the labeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = dsp.labeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['explanatory','groupby'], 'year':'trend', 'driver_gender':['explanatory','groupby'], 'driver_race':['explanatory','groupby'],
       'decriminalization':['explanatory','groupby'], 'medical':['explanatory','groupby'],
         'recreational':['explanatory','groupby'], 'no_reforms':['explanatory','groupby'],
       'search_conducted_false':'trend', 'search_conducted_true':'trend',
       'search_conducted_rate':'trend', 'contraband_found_false':'trend',
       'contraband_found_true':'trend', 'contraband_found_rate':'trend', 'hit_false':'trend',
       'hit_true':'trend', 'hit_rate':'trend', 'num_stops':'trend'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.meta_df

/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['role'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[explanatory, groupby]",False,NaN
year,int64,ordinal,trend,False,NaN
driver_gender,object,binary,"[explanatory, groupby]",False,NaN
driver_race,object,categorical,"[explanatory, groupby]",False,NaN
decriminalization,int64,binary,"[explanatory, groupby]",False,NaN
medical,int64,binary,"[explanatory, groupby]",False,NaN
recreational,int64,binary,"[explanatory, groupby]",False,NaN
no_reforms,int64,binary,"[explanatory, groupby]",False,NaN
search_conducted_false,float64,continuous,trend,True,NaN


In [7]:
labeled_df_setup.meta_df.index.name

'variable'

Now, we've set this up, we can also save these configurations to load them in directly in the future

In [8]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

We can see what this does, using a bash magic

In [9]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [10]:
labeled_df = dsp.labeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[explanatory, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,binary,"[explanatory, groupby]",False,NaN
driver_race,object,categorical,"[explanatory, groupby]",False,NaN
decriminalization,int64,binary,"[explanatory, groupby]",False,NaN
medical,int64,binary,"[explanatory, groupby]",False,NaN
recreational,int64,binary,"[explanatory, groupby]",False,NaN
no_reforms,int64,binary,"[explanatory, groupby]",False,NaN
search_conducted_false,float64,continuous,[trend],True,NaN


In [11]:
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [12]:
corrobj = dsp.all_pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

Index(['year', 'search_conducted_false', 'search_conducted_true',
       'search_conducted_rate', 'contraband_found_false',
       'contraband_found_true', 'contraband_found_rate', 'hit_false',
       'hit_true', 'hit_rate', 'num_stops'],
      dtype='object', name='variable')

In [13]:
rankobj = dsp.mean_rank_trend()
linreg_obj = dsp.linear_trend()

# Computing Trends on a labeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [14]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

,feat1,feat2,subgroup_trend,group_feat,subgroup,trend_type,agg_trend
0,year,search_conducted_false,0.299408,state,AZ,pearson_corr,-0.043351
1,year,search_conducted_false,-0.165978,state,CO,pearson_corr,-0.043351
2,year,search_conducted_false,-0.052226,state,CT,pearson_corr,-0.043351
3,year,search_conducted_false,-0.084114,state,IL,pearson_corr,-0.043351
4,year,search_conducted_false,0.118854,state,MA,pearson_corr,-0.043351


In [15]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

,feat1,feat2,subgroup_trend,subgroup,group_feat,trend_type,agg_trend
1531,hit_false,num_stops,29.063,IL,state,lin_reg,NaN
935,search_conducted_rate,contraband_found_rate,NaN,1,medical,lin_reg,NaN
974,search_conducted_rate,hit_true,NaN,NC,state,lin_reg,NaN
449,search_conducted_false,contraband_found_false,1.00895,F,driver_gender,lin_reg,1.01613
13,search_conducted_false,driver_race,"[Other, Asian, Hispanic, Black, White]",F,driver_gender,rank_trend,"[Other, Asian, Black, Hispanic, White]"
1248,contraband_found_true,hit_false,NaN,AZ,state,lin_reg,NaN
1447,contraband_found_rate,num_stops,NaN,IL,state,lin_reg,NaN
1497,hit_false,hit_true,NaN,1,recreational,lin_reg,NaN
1038,search_conducted_rate,num_stops,-408674,M,driver_gender,lin_reg,-257201
1023,search_conducted_rate,hit_rate,NaN,1,no_reforms,lin_reg,NaN


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

In [16]:
pd.unique(labeled_df.result_df['trend_type'])

array(['rank_trend', 'lin_reg'], dtype=object)

In [17]:
labeled_df.trend_list

In [18]:
def dist_helper(row):
    trend_dist = {t.name:t.get_distance for t in labeled_df.trend_list}
    
    row['distance'] = trend_dist[row['trend_type']](row)
    return row['distance']
#     return trend_dist[row['trend_type']]()
    

In [19]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance()

In [20]:
labeled_df.result_df

,feat1,feat2,subgroup_trend,subgroup,group_feat,trend_type,agg_trend,distance
0,search_conducted_false,driver_race,"[Asian, Black, Other, Hispanic, White]",AZ,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.40
1,search_conducted_false,driver_race,"[Other, Asian, Black, Hispanic, White]",CO,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.00
2,search_conducted_false,driver_race,"[Other, Asian, Hispanic, Black, White]",CT,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.20
3,search_conducted_false,driver_race,"[Other, Asian, Hispanic, Black, White]",IL,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.20
4,search_conducted_false,driver_race,"[Other, Asian, Hispanic, Black, White]",MA,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.20
5,search_conducted_false,driver_race,"[Asian, Other, Hispanic, Black, White]",MD,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.40
6,search_conducted_false,driver_race,"[Asian, Other, Hispanic, Black, White]",NC,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.40
7,search_conducted_false,driver_race,"[Other, Asian, Hispanic, Black, White]",RI,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.20
8,search_conducted_false,driver_race,"[Other, Hispanic, Black, White]",SC,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.26
9,search_conducted_false,driver_race,"[Other, Asian, Black, Hispanic, White]",TX,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.00


In [21]:
n_s = 3
n_a = 5
list(range(n_s,n_a))

[3, 4]

In [22]:
labeled_df.result_df

,feat1,feat2,subgroup_trend,subgroup,group_feat,trend_type,agg_trend,distance
0,search_conducted_false,driver_race,"[Asian, Black, Other, Hispanic, White]",AZ,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.40
1,search_conducted_false,driver_race,"[Other, Asian, Black, Hispanic, White]",CO,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.00
2,search_conducted_false,driver_race,"[Other, Asian, Hispanic, Black, White]",CT,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.20
3,search_conducted_false,driver_race,"[Other, Asian, Hispanic, Black, White]",IL,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.20
4,search_conducted_false,driver_race,"[Other, Asian, Hispanic, Black, White]",MA,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.20
5,search_conducted_false,driver_race,"[Asian, Other, Hispanic, Black, White]",MD,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.40
6,search_conducted_false,driver_race,"[Asian, Other, Hispanic, Black, White]",NC,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.40
7,search_conducted_false,driver_race,"[Other, Asian, Hispanic, Black, White]",RI,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.20
8,search_conducted_false,driver_race,"[Other, Hispanic, Black, White]",SC,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.26
9,search_conducted_false,driver_race,"[Other, Asian, Black, Hispanic, White]",TX,state,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.00


In [23]:
np.round()

TypeError: round_() missing 1 required positional argument: 'a'